In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
baseMapNum = 32
weight_decay = 1e-4

act, act_name = "elu", "elu"
print(act_name)


model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

elu
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
______

In [3]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [4]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 150s - loss: 2.0000 - acc: 0.4003 - val_loss: 1.2756 - val_acc: 0.5809
Epoch 2/75
390/390 [==============================] - 146s - loss: 1.3452 - acc: 0.5663 - val_loss: 1.0213 - val_acc: 0.6612
Epoch 3/75
390/390 [==============================] - 145s - loss: 1.1550 - acc: 0.6271 - val_loss: 0.8808 - val_acc: 0.7187
Epoch 4/75
390/390 [==============================] - 145s - loss: 1.0330 - acc: 0.6665 - val_loss: 0.8787 - val_acc: 0.7343
Epoch 5/75
390/390 [==============================] - 145s - loss: 0.9604 - acc: 0.6900 - val_loss: 0.8497 - val_acc: 0.7369
Epoch 6/75
390/390 [==============================] - 144s - loss: 0.9011 - acc: 0.7148 - val_loss: 0.7278 - val_acc: 0.7696
Epoch 7/75
390/390 [==============================] - 144s - loss: 0.8629 - acc: 0.7277 - val_loss: 0.7254 - val_acc: 0.7826
Epoch 8/75
390/390 [==============================] - 144s - loss: 0.8293 - acc: 0.7398 - val_loss: 0.6596 - val_acc: 0.8051


390/390 [==============================] - 145s - loss: 0.5555 - acc: 0.8543 - val_loss: 0.4779 - val_acc: 0.8811
Epoch 67/75
390/390 [==============================] - 144s - loss: 0.5564 - acc: 0.8534 - val_loss: 0.4847 - val_acc: 0.8787
Epoch 68/75
390/390 [==============================] - 144s - loss: 0.5599 - acc: 0.8512 - val_loss: 0.4975 - val_acc: 0.8769
Epoch 69/75
390/390 [==============================] - 144s - loss: 0.5549 - acc: 0.8518 - val_loss: 0.5069 - val_acc: 0.8750
Epoch 70/75
390/390 [==============================] - 144s - loss: 0.5546 - acc: 0.8534 - val_loss: 0.5100 - val_acc: 0.8732
Epoch 71/75
390/390 [==============================] - 144s - loss: 0.5569 - acc: 0.8516 - val_loss: 0.4740 - val_acc: 0.8809
Epoch 72/75
390/390 [==============================] - 144s - loss: 0.5538 - acc: 0.8530 - val_loss: 0.4925 - val_acc: 0.8750
Epoch 73/75
390/390 [==============================] - 144s - loss: 0.5515 - acc: 0.8543 - val_loss: 0.4988 - val_acc: 0.8760
Epoc

In [5]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 145s - loss: 0.5143 - acc: 0.8684 - val_loss: 0.4442 - val_acc: 0.8918
Epoch 2/25
390/390 [==============================] - 144s - loss: 0.5016 - acc: 0.8704 - val_loss: 0.4430 - val_acc: 0.8942
Epoch 3/25
390/390 [==============================] - 144s - loss: 0.4926 - acc: 0.8720 - val_loss: 0.4419 - val_acc: 0.8918
Epoch 4/25
390/390 [==============================] - 144s - loss: 0.4883 - acc: 0.8714 - val_loss: 0.4331 - val_acc: 0.8953
Epoch 5/25
390/390 [==============================] - 144s - loss: 0.4854 - acc: 0.8748 - val_loss: 0.4411 - val_acc: 0.8953
Epoch 6/25
390/390 [==============================] - 144s - loss: 0.4805 - acc: 0.8761 - val_loss: 0.4323 - val_acc: 0.8959
Epoch 7/25
390/390 [==============================] - 144s - loss: 0.4801 - acc: 0.8754 - val_loss: 0.4325 - val_acc: 0.8965
Epoch 8/25
390/390 [==============================] - 144s - loss: 0.4738 - acc: 0.8757 - val_loss: 0.4279 - val_acc: 0.8954


In [6]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 145s - loss: 0.4287 - acc: 0.8871 - val_loss: 0.4106 - val_acc: 0.9000
Epoch 2/25
390/390 [==============================] - 144s - loss: 0.4255 - acc: 0.8881 - val_loss: 0.4011 - val_acc: 0.8995
Epoch 3/25
390/390 [==============================] - 144s - loss: 0.4241 - acc: 0.8894 - val_loss: 0.4021 - val_acc: 0.8995
Epoch 4/25
390/390 [==============================] - 144s - loss: 0.4199 - acc: 0.8898 - val_loss: 0.3929 - val_acc: 0.9006
Epoch 5/25
390/390 [==============================] - 144s - loss: 0.4139 - acc: 0.8911 - val_loss: 0.3921 - val_acc: 0.9027
Epoch 6/25
390/390 [==============================] - 144s - loss: 0.4165 - acc: 0.8892 - val_loss: 0.3947 - val_acc: 0.9020
Epoch 7/25
390/390 [==============================] - 144s - loss: 0.4111 - acc: 0.8906 - val_loss: 0.4055 - val_acc: 0.8982
Epoch 8/25
390/390 [==============================] - 144s - loss: 0.4097 - acc: 0.8926 - val_loss: 0.4050 - val_acc: 0.8970


In [7]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 90.200 loss: 0.381


In [17]:
baseMapNum = 32
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.load_weights("models/cifar10_normal_rms_ep125elu.h5")

In [18]:
# Continue for 25 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.00015,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep150'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 145s - loss: 0.3850 - acc: 0.8969 - val_loss: 0.3781 - val_acc: 0.9039
Epoch 2/25
390/390 [==============================] - 144s - loss: 0.3770 - acc: 0.9001 - val_loss: 0.3763 - val_acc: 0.9039
Epoch 3/25
390/390 [==============================] - 144s - loss: 0.3713 - acc: 0.9020 - val_loss: 0.3741 - val_acc: 0.9060
Epoch 4/25
390/390 [==============================] - 144s - loss: 0.3667 - acc: 0.9029 - val_loss: 0.3736 - val_acc: 0.9074
Epoch 5/25
390/390 [==============================] - 144s - loss: 0.3694 - acc: 0.9024 - val_loss: 0.3798 - val_acc: 0.9048
Epoch 6/25
390/390 [==============================] - 144s - loss: 0.3664 - acc: 0.9036 - val_loss: 0.3756 - val_acc: 0.9052
Epoch 7/25
390/390 [==============================] - 144s - loss: 0.3667 - acc: 0.9031 - val_loss: 0.3764 - val_acc: 0.9033
Epoch 8/25
390/390 [==============================] - 144s - loss: 0.3684 - acc: 0.9020 - val_loss: 0.3688 - val_acc: 0.9069


In [19]:
print(his.history)

{'acc': [0.89689503205128207, 0.9000641642987518, 0.90198909205659139, 0.90285129934539765, 0.90243022138582119, 0.90365335254436663, 0.90313201800423781, 0.90198909207571387, 0.90230991341649325, 0.90355309596381439, 0.90383381460353196, 0.90407443054218806, 0.90421478982380199, 0.90305181260211442, 0.90341273662483301, 0.90371350661508154, 0.9020692973631026, 0.90642043629784896, 0.90441530313788598, 0.90583894772525009, 0.90407443050394309, 0.90194898941289703, 0.90575874234224918, 0.90481632978517657, 0.90367340395226481], 'val_loss': [0.37810574879646303, 0.37627739319801329, 0.37409518642425538, 0.37360683350563051, 0.37978106174468995, 0.37556017384529111, 0.37636747665405273, 0.36877086877822873, 0.37507552075386047, 0.3720274420261383, 0.37201234216690066, 0.37461900806427001, 0.3679748847961426, 0.37166594042778017, 0.37686232585906981, 0.36433514051437377, 0.37217071504592897, 0.37146086997985839, 0.37311287226676942, 0.37386523008346556, 0.3708099145889282, 0.37034315271377

In [20]:
# Continue for 25 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000075,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep175'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 145s - loss: 0.3418 - acc: 0.9090 - val_loss: 0.3619 - val_acc: 0.9075
Epoch 2/25
390/390 [==============================] - 143s - loss: 0.3443 - acc: 0.9081 - val_loss: 0.3609 - val_acc: 0.9078
Epoch 3/25
390/390 [==============================] - 144s - loss: 0.3423 - acc: 0.9089 - val_loss: 0.3585 - val_acc: 0.9077
Epoch 4/25
390/390 [==============================] - 144s - loss: 0.3408 - acc: 0.9094 - val_loss: 0.3645 - val_acc: 0.9085
Epoch 5/25
390/390 [==============================] - 143s - loss: 0.3384 - acc: 0.9098 - val_loss: 0.3577 - val_acc: 0.9096
Epoch 6/25
390/390 [==============================] - 144s - loss: 0.3405 - acc: 0.9087 - val_loss: 0.3637 - val_acc: 0.9072
Epoch 7/25
390/390 [==============================] - 144s - loss: 0.3375 - acc: 0.9110 - val_loss: 0.3628 - val_acc: 0.9071
Epoch 8/25
390/390 [==============================] - 144s - loss: 0.3439 - acc: 0.9086 - val_loss: 0.3651 - val_acc: 0.9062


In [21]:
print(his.history)

{'acc': [0.90897435897435896, 0.90812479950580838, 0.90892685270477724, 0.90938803332717055, 0.90980911132499198, 0.9086461341224269, 0.91095203725351592, 0.90858598008995684, 0.90914741738851457, 0.91167388510772884, 0.91081167793365714, 0.90988931666974804, 0.91065126726326751, 0.90880654471632683, 0.912155117080653, 0.91367901824856934, 0.91079162658312629, 0.90968880335566393, 0.91157362850805412, 0.91145332048135874, 0.91243583573949305, 0.91167388516509618, 0.91012993266577136, 0.91253609237741273, 0.91231552771279778], 'val_loss': [0.36187692437171937, 0.36088648214340208, 0.35854447045326232, 0.36446169748306273, 0.35768763680458071, 0.36367765750885012, 0.36282546310424807, 0.36511099023818971, 0.36076046829223635, 0.36497328929901124, 0.36386575880050659, 0.36292336893081667, 0.3604862789154053, 0.3636804880142212, 0.36379329409599304, 0.35955747680664063, 0.35565124478340149, 0.35730977706909178, 0.3595226854324341, 0.35796147594451905, 0.3595506537437439, 0.3599053977489471

In [22]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000035,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep180'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 148s - loss: 0.3253 - acc: 0.9133 - val_loss: 0.3552 - val_acc: 0.9095
Epoch 2/5
390/390 [==============================] - 145s - loss: 0.3194 - acc: 0.9153 - val_loss: 0.3544 - val_acc: 0.9100
Epoch 3/5
390/390 [==============================] - 145s - loss: 0.3254 - acc: 0.9144 - val_loss: 0.3541 - val_acc: 0.9099
Epoch 4/5
390/390 [==============================] - 145s - loss: 0.3235 - acc: 0.9137 - val_loss: 0.3564 - val_acc: 0.9090
Epoch 5/5
390/390 [==============================] - 145s - loss: 0.3229 - acc: 0.9128 - val_loss: 0.3559 - val_acc: 0.9086


In [23]:
print(his.history)

{'acc': [0.91332131410256412, 0.91528312476124185, 0.91440086625576167, 0.91367901826769182, 0.91281681103625278], 'val_loss': [0.35519334955215454, 0.35437487621307373, 0.35411906323432923, 0.35642164855003355, 0.35585872044563294], 'loss': [0.32531605038123251, 0.31935959300973454, 0.32541894047429326, 0.32343480167841737, 0.32285918094119775], 'val_acc': [0.90949999999999998, 0.91000000000000003, 0.90990000000000004, 0.90900000000000003, 0.90859999999999996]}


In [24]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000017,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep185'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 145s - loss: 0.3269 - acc: 0.9126 - val_loss: 0.3538 - val_acc: 0.9094
Epoch 2/5
390/390 [==============================] - 144s - loss: 0.3184 - acc: 0.9141 - val_loss: 0.3514 - val_acc: 0.9102
Epoch 3/5
390/390 [==============================] - 144s - loss: 0.3157 - acc: 0.9165 - val_loss: 0.3531 - val_acc: 0.9099
Epoch 4/5
390/390 [==============================] - 144s - loss: 0.3233 - acc: 0.9138 - val_loss: 0.3534 - val_acc: 0.9095
Epoch 5/5
390/390 [==============================] - 144s - loss: 0.3233 - acc: 0.9141 - val_loss: 0.3529 - val_acc: 0.9101


In [25]:
print(his.history)

{'acc': [0.91264022435897441, 0.9141201476160441, 0.91652630736593022, 0.91381937762579557, 0.91405999358357393], 'val_loss': [0.3538467577934265, 0.35142908020019531, 0.35314376525878904, 0.35344503831863405, 0.35289660987854005], 'loss': [0.32687588570973813, 0.31842460483551638, 0.31569306115940438, 0.32334118961025821, 0.3232502385634507], 'val_acc': [0.90939999999999999, 0.91020000000000001, 0.90990000000000004, 0.90949999999999998, 0.91010000000000002]}


In [26]:
# Continue for 5 epochs more
batch_size = 128
epochs=25

opt_rms = keras.optimizers.rmsprop(lr=0.000012,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs/5,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep190'+act_name+'.h5')

Epoch 1/5
390/390 [==============================] - 145s - loss: 0.3217 - acc: 0.9155 - val_loss: 0.3536 - val_acc: 0.9100
Epoch 2/5
390/390 [==============================] - 144s - loss: 0.3211 - acc: 0.9151 - val_loss: 0.3530 - val_acc: 0.9090
Epoch 3/5
390/390 [==============================] - 144s - loss: 0.3178 - acc: 0.9158 - val_loss: 0.3531 - val_acc: 0.9101
Epoch 4/5
390/390 [==============================] - 144s - loss: 0.3147 - acc: 0.9171 - val_loss: 0.3530 - val_acc: 0.9094
Epoch 5/5
390/390 [==============================] - 144s - loss: 0.3195 - acc: 0.9155 - val_loss: 0.3523 - val_acc: 0.9098


In [27]:
print(his.history)

{'acc': [0.91548477564102559, 0.91506256015399423, 0.91582451078575855, 0.91708774458799835, 0.91550368946410154], 'val_loss': [0.35359545803070069, 0.35298236584663389, 0.35310207810401917, 0.35297208704948424, 0.35232336378097534], 'loss': [0.32171604285637539, 0.32113615497779419, 0.31780275674748965, 0.3146676657136373, 0.31954576012451069], 'val_acc': [0.91000000000000003, 0.90900000000000003, 0.91010000000000002, 0.90939999999999999, 0.90980000000000005]}
